## 가설설정
- 1. 국가별/상품군별 온라인 쇼핑 해외직접판매는 성장하고 있을까?
- 2. 해외 직접판매를 한다면 어느 국가에 대해, 어느 상품으로 판매전략을 세우면 좋을까?

## 1. 패키지 임포트 및 데이터 가져오기

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

plt.rc('font', family="Malgun Gothic")
plt.rc('axes', unicode_minus=False)

In [21]:
# 레티나 설정을 해주면 글씨가 좀 더 선명하게 보입니다.
# 폰트의 주변이 흐릿하게 보이는 것을 방지합니다.
%config InlineBackend.figure_format = 'retina'

In [22]:
df_raw=pd.read_csv("국가_대륙_별_상품군별_온라인쇼핑_해외직접판매액_20220327225120.csv", encoding='cp949')
df_raw.shape

(450, 26)

In [23]:
df_raw.head()

,국가(대륙)별,상품군별,판매유형별,2014.1/4,2014.2/4,2014.3/4,2014.4/4,2015.1/4,2015.2/4,2015.3/4,...,2017.2/4,2017.3/4,2017.4/4,2018.1/4,2018.2/4,2018.3/4,2018.4/4,2019.1/4,2019.2/4,2019.3/4
0,합계,합계,계,148272,153641,163999,213216,269101,271784,268421,...,574551,749612,858240,833672,897205,921586,974060,1251122,1345723,1526135
1,합계,합계,면세점,-,-,-,-,-,-,-,...,441096,610630,677884,665613,725319,761836,770656,1070693,1157158,1336372
2,합계,합계,면세점 이외,-,-,-,-,-,-,-,...,133455,138982,180356,168059,171886,159750,203404,180429,188565,189763
3,합계,컴퓨터 및 주변기기,계,4915,4052,3912,3529,2903,2697,3804,...,2043,2018,2158,5236,3854,4320,4511,3702,4038,3670
4,합계,컴퓨터 및 주변기기,면세점,-,-,-,-,-,-,-,...,3,5,1,2,6,1,215,2,0,1


In [24]:
#국가(대륙)별 데이터 빈도수 세기
df_raw["국가(대륙)별"].value_counts()

합계            45
미국            45
중국            45
일본            45
아세안(ASEAN)    45
유럽연합(EU)      45
중동            45
중남미           45
대양주           45
기타            45
Name: 국가(대륙)별, dtype: int64

In [25]:
# 미국데이터만 따로 보기
df_raw[df_raw["국가(대륙)별"]=="미국"].head()

,국가(대륙)별,상품군별,판매유형별,2014.1/4,2014.2/4,2014.3/4,2014.4/4,2015.1/4,2015.2/4,2015.3/4,...,2017.2/4,2017.3/4,2017.4/4,2018.1/4,2018.2/4,2018.3/4,2018.4/4,2019.1/4,2019.2/4,2019.3/4
45,미국,합계,계,25773,22573,21288,26729,39945,26654,28647,...,40976,41927,53209,42679,40207,37631,48978,40985,44973,55512
46,미국,합계,면세점,-,-,-,-,-,-,-,...,9183,9469,12177,5184,3603,2317,2079,2657,2951,1818
47,미국,합계,면세점 이외,-,-,-,-,-,-,-,...,31793,32458,41032,37495,36604,35314,46899,38328,42022,53694
48,미국,컴퓨터 및 주변기기,계,2216,1666,1776,1675,1469,1393,2166,...,946,791,854,2033,1553,1784,1774,1298,1408,1653
49,미국,컴퓨터 및 주변기기,면세점,-,-,-,-,-,-,-,...,0,0,0,0,0,0,1,0,0,0


In [26]:
#"분기"가 현재 컬럼명으로 지정되어 있기때문에 melt함수로 데이터를 행으로 바꿔줌
df=df_raw.melt(id_vars=["국가(대륙)별", "상품군별", "판매유형별"], var_name="기간", value_name="백만원")
print(df.shape)
print(df.head())
print(df.info())

(10350, 5)
  국가(대륙)별        상품군별   판매유형별        기간     백만원
0      합계          합계       계  2014.1/4  148272
1      합계          합계     면세점  2014.1/4       -
2      합계          합계  면세점 이외  2014.1/4       -
3      합계  컴퓨터 및 주변기기       계  2014.1/4    4915
4      합계  컴퓨터 및 주변기기     면세점  2014.1/4       -
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10350 entries, 0 to 10349
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   국가(대륙)별  10350 non-null  object
 1   상품군별     10350 non-null  object
 2   판매유형별    10350 non-null  object
 3   기간       10350 non-null  object
 4   백만원      10350 non-null  object
dtypes: object(5)
memory usage: 404.4+ KB
None


## 2. 전처리

In [27]:
#기간에서 연도, 분기를 분리
#기간에서 연도 분리
df["연도"]=df["기간"].map(lambda x : int(x,split()[0]))
df.head()

#기간에서 분기 분리
df["분기"]=df["기간"].map(lambda x : int(x.split()[1].split("/")[0]))
df.head()

NameError: name 'split' is not defined